In [9]:
import tensorflow as tf
from keras.models import load_model
from pathlib import Path
from art_api import utils, config
import os
from PIL import Image
import numpy as np
import pandas as pd
import wandb
from wandb.keras import WandbCallback
from abc import ABC, abstractmethod
from tensorflow.keras import Sequential, layers, models
print(f"memory usage {tf.config.experimental.get_memory_info('GPU:0')['current'] / 10 ** 9} GB")

%load_ext autoreload

memory usage 0.0 GB


2022-09-19 02:15:56.346923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-19 02:15:56.425487: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-19 02:15:56.425781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-19 02:15:56.428020: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [2]:
print(config.wandb.config)

{'LEARNING_RATE': 0.0001, 'N_EPOCHS': 50, 'BATCH_SIZE': 32, 'DROPOUT_RATE': 0.0, 'IM_SIZE': 256, 'REGULARIZATION_RATE': 0.0, 'N_FILTERS': 6, 'KERNEL_SIZE': 3, 'N_STRIDES': 1, 'POOL_SIZE': 2, 'N_DENSE_1': 100, 'N_DENSE_2': 10}


In [ ]:
lenet_model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),

    Conv2D(filters = N_FILTERS , kernel_size = KERNEL_SIZE, strides = N_STRIDES , padding='valid',
          activation = 'relu',kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),
    MaxPool2D (pool_size = POOL_SIZE, strides= N_STRIDES*2),
    Dropout(rate = DROPOUT_RATE ),

    Conv2D(filters = N_FILTERS*2 + 4, kernel_size = KERNEL_SIZE, strides=N_STRIDES, padding='valid',
          activation = 'relu', kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),
    MaxPool2D (pool_size = POOL_SIZE, strides= N_STRIDES*2),

    Flatten(),
    
    Dense( CONFIGURATION['N_DENSE_1'], activation = "relu", kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),
    Dropout(rate = DROPOUT_RATE),
    
    Dense( CONFIGURATION['N_DENSE_2'], activation = "relu", kernel_regularizer = L2(REGULARIZATION_RATE)),
    BatchNormalization(),

    Dense(1, activation = "sigmoid"),

])

lenet_model.summary()

In [15]:
class Model:
    def __init__(self, datafile = config.DF_PATH):
        self.df = pd.read_csv(datafile)
        #self.model = load_baseline_model()
        
    def load_baseline_model():
        model = Sequential()
        model.add(Rescaling(1./255, input_shape = (config.IM_SIZE, config.IM_SIZE, 3)))

        model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
        model.add(layers.MaxPooling2D(3))

        model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
        model.add(layers.MaxPooling2D(3))

        model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
        model.add(layers.MaxPooling2D(3))

        model.add(layers.Flatten())
        model.add(layers.Dense(100, activation='relu'))
        model.add(layers.Dense(10, activation='sigmoid'))

        opt = optimizers.Adam(learning_rate=1e-4)
        model.compile(loss='binary_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])
        
        model.summary()

        return model
    
    def split(self, test_size):
        X = np.array(self.df[['Humidity', 'Pressure (millibars)']])
        y = np.array(self.df['Temperature (C)'])
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = test_size, random_state = 42)
    
    def fit(self):
        self.model = self.linear_reg.fit(self.X_train, self.y_train)
    
    def predict(self):
        result = self.linear_reg.predict(self.X_test)
        return result
    
    def preproc(self):
        '''Preprocess for the chosen model'''
    
    def build_model(model):
        '''model can be baseline, vgg16, vgg19, resnet50, ResNet152, ResNet152V2, InceptionV3, InceptionResNetV2, Xception'''
        model = load_resnet152_model()
        model = add_last_layers(model)

        model.compile(loss='binary_crossentropy',
                      optimizer=optimizers.Adam(learning_rate=1e-4),
                      metrics=['accuracy'])
        return model

if __name__ == '__main__':
    model_instance = Model()
    #model_instance.split(0.2)
    #model_instance.fit() 
    #print(model_instance.predict())   
    #print("Accuracy: ",     model_instance.model.score(model_instance.X_test, model_instance.y_test))

In [3]:
class BaseModel:

    def __init__(self, cfg):
        self.config = config.wandb.config

    def load_local(self):
        """Loads dataset from local path"""
        df = pd.read_csv(local_path)
        return df

In [4]:
print(config.DF_PATH)

../raw_data/df_yourpaintings.csv


In [5]:
baseline = BaseModel(config.wandb.config)
print(baseline)

In [6]:
df = baseline.load_data(config.DF_PATH)

AttributeError: 'BaseModel' object has no attribute 'load_data'

In [23]:
class BaseModel(ABC):
    '''Abstract Model class that is inherited to all models
    Behaviours:
    Get X and y
    Resize, load as array
    Preprocess input
    Get_config: hyperparameter tuning
    Model: baseline, VGG16, VGG19, ResNet, Inception, Xception
    Unfreeze layer
    Compile
    Fit
    Evaluate: accuracy, classification report, confusion matrix
    Predict with different thresholds (0.5, mean, median)
    Save model
    Df to csv
    '''
    def __init__(self, cfg):
        self.config = config.wandb.config

    @abstractmethod
    def load_data(self):
        self.df=DataLoader.load_cloud()
        return df

    @abstractmethod
    def build(self):
        pass

    @abstractmethod
    def train(self):
        pass

    @abstractmethod
    def evaluate(self):
        pass

In [22]:
baseline = BaseModel(config.wandb.config)

TypeError: Can't instantiate abstract class BaseModel with abstract methods build, evaluate, load_data, train

In [16]:
df = baseline.load_data()

TypeError: load_data() missing 1 required positional argument: 'self'

In [ ]:
def load_baseline_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(256,256,3)))

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(10, activation='sigmoid'))
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

In [5]:
MODEL_PATH = Path("../models/vgg16_bal")
PATH_NGS = Path("../raw_data/aws10k")
PATH_NGS_SM = Path("../raw_data/aws10k_sm")

In [4]:
model = load_model(MODEL_PATH)

2022-09-17 13:41:49.242183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
X, y = utils.load_data()

_request non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object., 401
Traceback (most recent call last):
  File "/Users/enjiaochen/.pyenv/versions/lewagon/lib/python3.8/site-packages/gcsfs/retry.py", line 115, in retry_request
    return await func(*args, **kwargs)
  File "/Users/enjiaochen/.pyenv/versions/lewagon/lib/python3.8/site-packages/gcsfs/core.py", line 384, in _request
    validate_response(status, contents, path, args)
  File "/Users/enjiaochen/.pyenv/versions/lewagon/lib/python3.8/site-packages/gcsfs/retry.py", line 102, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object., 401


HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object., 401

In [40]:
df = pd.read_csv(Path("../raw_data/data_raw_data_df_yourpaintings.csv"))

In [12]:
def resize_rescale(source):
    '''Download images and resize to 256x256 if not already in local disk
    Args:
        source: df, local, cloud
    Returns:
    '''
#    df = pd.read_csv(config.PATH_FILE)
    if source == "df":
        for index, row in df.iterrows():
            img_file = str(row["filename"])
            image = Image.open(os.path.join(config.PATH_YOURPAINTINGS, img_file))
            image = image.resize((256, 256), Image.ANTIALIAS)
            image.save(os.path.join(config.PATH_YOURPAINTINGS_SM, img_file))
            #imgs.append(np.array(image/255))
    if source == "path":
        for file in os.listdir(PATH_NGS):
            image = Image.open(os.path.join(PATH_NGS, file))
            image = image.resize((256, 256), Image.ANTIALIAS)
            image.save(os.path.join(PATH_NGS_SM, file))
    return "Images resized and rescaled"

In [41]:
resize_rescale(source="df")

/var/folders/11/0smvrnys6yn2p6mvqskf9vxm0000gn/T/ipykernel_12159/27729670.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((256, 256), Image.ANTIALIAS)


'Images resized and rescaled'

In [18]:
def load_data():
    '''generates X and y'''
    '''read from disk or read from cloud <--- to be implemented'''
    
    imgs = []

    for file in os.listdir(PATH_NGS):
        image = Image.open(os.path.join(PATH_NGS_SM, file))   
        imgs.append(np.array(image))
        X = np.array(imgs)
        X.shape
    return X

In [21]:
X = load_data()
X = np.array(X)

In [20]:
X.shape

(10, 256, 256, 3)

In [22]:
pred = model.predict(X)

1/1 [==============================] - 2s 2s/step


In [24]:
pred[0]

array([0.24459508, 0.12576418, 0.06244959, 0.00660372, 0.04267118,
       0.01717522, 0.0298612 , 0.01034015, 0.08589564, 0.00666896],
      dtype=float32)

# Generate confusion matrix for model

In [37]:
df = pd.read_csv(Path("../raw_data/data_raw_data_df_yourpaintings.csv"))
df.columns

Index(['index', 'Image URL', 'Web page URL', 'Subset', 'Labels', 'filename',
       'labels', 'aeroplane', 'bird', 'boat', 'chair', 'cow', 'diningtable',
       'dog', 'horse', 'sheep', 'train'],
      dtype='object')

In [42]:
def load_data():
    '''generates X and y'''
    '''read from disk or read from cloud <--- to be implemented'''
    df = pd.read_csv(Path("../raw_data/data_raw_data_df_yourpaintings.csv"))
    df
    imgs = []
    for index, row in df.iterrows():
        img_file = str(row["filename"])
        image = Image.open(os.path.join(config.PATH_YOURPAINTINGS_SM, img_file))   
        imgs.append(np.array(image))
        X = np.array(imgs)
        X.shape
        y = df.drop(columns=['index', 'Image URL', 'Web page URL', 'Subset', 'Labels', 'filename', 'labels'])
        y.shape
    return X, y

In [43]:
X, y = load_data()